##  Cynaptics  Induction Task sub task 1
### This code is for the second kaggle compitition i found this model to do better in new datset than prev model in this i used  Resnet and added some layers on top of it .i also unfrezzed some layers of the resnet model  accuracy on kaggle test is around 0.63 


In [ ]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(150, 150, 3)
)


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

model = Sequential()
model.add(conv_base)  
model.add(Flatten()) 
model.add(Dense(256, activation='relu'))  
model.add(Dense(1, activation='sigmoid'))  


In [ ]:
len(model.trainable_weights)


In [ ]:

for layer in conv_base.layers:
    layer.trainable = False


len(model.trainable_weights)


In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [22]:
validation_dir = 'Data/Validation'
train_dir = 'Data/Train'



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,  
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,  
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(learning_rate=2e-5),
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size  
)



In [9]:
unfreeze = False  

for layer in conv_base.layers:
    if layer.name == 'block3_conv1':
        unfreeze = True  
    if unfreeze:
        layer.trainable = True  
    else:
        layer.trainable = False  


In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

for layer in conv_base.layers:
    layer.trainable = False  

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  
model.add(Dropout(0.5))
model.add(BatchNormalization())  
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(learning_rate=1e-5),
    metrics=['accuracy']
)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping, reduce_lr]
)


In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model  

test_dir = '/Users/aniket/cynaptics/Data/Test'


def safe_extract_number(filename):
    try:
        return int(filename.split('_')[-1].split('.')[0])
    except (IndexError, ValueError):
        return float('inf')

valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']

image_filenames = [f for f in os.listdir(test_dir) if any(f.lower().endswith(ext) for ext in valid_extensions)]
image_filenames = sorted(image_filenames, key=safe_extract_number)

predictions = []

for filename in image_filenames:
    img_path = os.path.join(test_dir, filename)

    if not any(filename.lower().endswith(ext) for ext in valid_extensions):
        print(f"Skipping non-image file: {filename}")
        continue

    try:
        img = image.load_img(img_path, target_size=(150, 150))  
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0 
        pred = model.predict(img_array)  

        
        predicted_class = 'Real' if pred[0] >= 0.5 else 'AI'

        
        filename_without_extension = os.path.splitext(filename)[0]

       
        predictions.append((filename_without_extension, predicted_class))

        print(f"Processed {filename} - Predicted: {predicted_class}")

    except Exception as e:
        print(f"Error processing image {filename}: {e}")
        continue


results_df = pd.DataFrame(predictions, columns=['Id', 'Label'])

submission_file_path = '/Users/aniket/cynaptics/Data/submission.csv' 
results_df.to_csv(submission_file_path, index=False)

print(f"\nSubmission saved to {submission_file_path}")
